In [1]:
folder_path = 'Indian Currency Dataset'

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import shutil
from PIL import Image
import numpy as np

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
train = os.path.join(folder_path, 'train')
test = os.path.join(folder_path, 'test')

In [5]:
train_fake = os.path.join(train, 'fake')
train_real = os.path.join(train, 'real')

In [6]:
train_fake_images = [os.path.join(train_fake, img) for img in os.listdir(train_fake)]
train_real_images = [os.path.join(train_real, img) for img in os.listdir(train_real)]

In [7]:
train_fake, val_fake = train_test_split(train_fake_images, test_size=0.15, random_state=42)
train_real, val_real = train_test_split(train_real_images, test_size=0.15, random_state=42)

In [8]:
train_images = train_fake + train_real
val_images = val_fake + val_real

In [9]:
validation = os.path.join(folder_path, 'validation')
os.makedirs(validation, exist_ok=True)

In [10]:
val_fake = os.path.join(validation, 'fake')
val_real = os.path.join(validation, 'real')
os.makedirs(val_fake, exist_ok=True)
os.makedirs(val_real, exist_ok=True)

In [11]:
import os
import shutil
import glob

# Define your actual source folder
source_folder = r"D:\AIR FORCE COMPUTER\Desktop\Projects\Face Detection\Fake-Currency-Checker\Indian Currency Dataset\validation\real"

# Collect all image file paths from that folder
image_files = glob.glob(os.path.join(source_folder, "*.jpg"))  # Change to *.png or *.jpeg if needed

# Destination folder (can be the same if you're cleaning or can be another folder)
destination_folder = r"D:\AIR FORCE COMPUTER\Desktop\Projects\Face Detection\Fake-Currency-Checker\Indian Currency Dataset\validation\real_cleaned"
os.makedirs(destination_folder, exist_ok=True)

# Move images
for img_path in image_files:
    if os.path.isfile(img_path):
        shutil.move(img_path, os.path.join(destination_folder, os.path.basename(img_path)))
    else:
        print(f"File not found: {img_path}")


In [12]:
print(f'Train set size: {len(train_images)}')
print(f'Validation set size: {len(val_images)}')

Train set size: 127
Validation set size: 23


In [13]:
batch_size = 32
target_size = (224, 224)


import os
import shutil
import glob

# Define your actual source folder
source_folder = r"D:\AIR FORCE COMPUTER\Desktop\Projects\Face Detection\Fake-Currency-Checker\Indian Currency Dataset\validation\real"

# Collect all image file paths from that folder
image_files = glob.glob(os.path.join(source_folder, "*.jpg"))  # Change to *.png or *.jpeg if needed

# Destination folder (can be the same if you're cleaning or can be another folder)
destination_folder = r"D:\AIR FORCE COMPUTER\Desktop\Projects\Face Detection\Fake-Currency-Checker\Indian Currency Dataset\validation\real_cleaned"
os.makedirs(destination_folder, exist_ok=True)

# Move images
for img_path in image_files:
    if os.path.isfile(img_path):
        shutil.move(img_path, os.path.join(destination_folder, os.path.basename(img_path)))
    else:
        print(f"File not found: {img_path}")
train_generator = datagen.flow_from_directory(
    train,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42
)

validation_generator = datagen.flow_from_directory(
    validation,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
    seed=42
)

test_generator = datagen.flow_from_directory(
    test,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
    seed=42
)

Found 150 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 107 images belonging to 2 classes.


In [14]:
print(os.listdir(train))
print(os.listdir(validation))
print(os.listdir(test))

['fake', 'real']
['fake', 'real']
['fake', 'real']


In [15]:
model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
model.trainable = False

In [16]:
flatten_layer = tf.keras.layers.Flatten()(model.output)
dropout_layer = tf.keras.layers.Dropout(0.5)(flatten_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(dropout_layer)

model = tf.keras.models.Model(model.input, output_layer)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
num_epochs = 10
model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

c:\Users\AIR FORCE COMPUTER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 46s 7s/step - accuracy: 0.4702 - loss: 3.1886 - val_accuracy: 0.7500 - val_loss: 1.6634
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 25s 3s/step - accuracy: 0.7316 - loss: 1.7360 - val_accuracy: 0.7833 - val_loss: 1.0984
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.8136 - loss: 0.7781 - val_accuracy: 0.9167 - val_loss: 0.3575
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - accuracy: 0.8119 - loss: 1.1594 - val_accuracy: 0.8167 - val_loss: 0.5596
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 4s/step - accuracy: 0.8380 - loss: 0.9909 - val_accuracy: 0.8000 - val_loss: 0.6815
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 16s 3s/step - accuracy: 0.8640 - loss: 0.6619 - val_accuracy: 0.9000 - val_loss: 0.3852
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.8511 - loss: 0.6340 - val_accuracy: 0.8500 - val_loss: 0.4616
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.8860 - loss: 0.3855 - val_accuracy: 0.9167 - val_loss: 0.2303
Epoch 9/

In [18]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.8991 - loss: 0.4748
Test Loss: 0.40755197405815125, Test Accuracy: 0.9065420627593994


In [19]:
train_loss, train_accuracy = model.evaluate(train_generator)
print(f'Train Loss: {train_loss}, Train Accuracy: {train_accuracy}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 2s/step - accuracy: 0.9077 - loss: 0.3926
Train Loss: 0.4036347568035126, Train Accuracy: 0.9133333563804626


In [20]:
predictions = model.predict(test_generator)

4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step  


In [21]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path)
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [22]:
def predict_currency(image_path, model):
    preprocessed_image = load_and_preprocess_image(image_path)
    prediction = model.predict(preprocessed_image)
    if prediction[0][0] >= 0.5:
       return "Real Currency"
    else:
       return "Fake Currency"

In [23]:
image_path = '/content/drive/My Drive/Indian Currency Dataset/2000.jpg'
result = predict_currency(image_path, model)
print(f"The predicted result is: {result}")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\content\\drive\\My Drive\\Indian Currency Dataset\\2000.jpg'

In [ ]:
image_path = '/content/drive/My Drive/Indian Currency Dataset/test (52).jpg'
result = predict_currency(image_path, model)
print(f"The predicted result is: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
The predicted result is: Fake Currency
